#### iLQR for point mass example with obstacle in 3d

In [1]:
import sys

sys.path.append('../')

import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
import time
from ocp import *
from costs import *
from ocp_utils import *

import pybullet as p
import pybullet_data

%load_ext autoreload
%autoreload 2
np.set_printoptions(precision=4, suppress=True)

## Steps:

#### 1. Define the dynamical system (e.g., double integrator, 2d robot, n-d manipulator, etc.)
#### 2. Define the cost functions
#### 3. Construct the ILQR problem
#### 4. Solve

## Setup Pybullet

In [2]:
def plot_traj(xs, obj_id, dt = 0.01):
    for x in xs:
        p.resetBasePositionAndOrientation(obj_id, x[:3], (0,0,0,1))
        time.sleep(dt)

In [3]:
p.connect(p.GUI)

p.setAdditionalSearchPath(pybullet_data.getDataPath())

In [5]:
p.resetSimulation()

#p.loadURDF('plane.urdf')
_,_,obj_id = create_primitives(rgbaColor=[0,0,1,1],radius = 0.04)
_,_,target_id = create_primitives(radius = 0.04, rgbaColor=[0,1,0,0.5])
_,_,border_id = create_primitives(rgbaColor=[1,0,0,0.1], shapeType=p.GEOM_BOX, halfExtents=[1.,1.,1.])

## Setup the System

#### Create point mass system with dimension 3 (double integrator)

In [6]:
dt = 0.05  #duration of 1 time step
T = 100    #total number of time steps
Dx, Du = 6, 3 #dimensions of x and u
dof = 3

#Define the matrix A and B to define a double integrator
A = np.eye(Dx)
A[:dof,dof:] = np.eye(dof)*dt
B = np.zeros((Dx, Du))
B[:dof,:] = 0.5*np.eye(Du)*(dt**2)
B[dof:, :] = np.eye(Du)*dt

sys = LinearSystem(A, B)

#### Set initial state and control trajectory

In [7]:
x0 = np.ones(Dx)*-1
sys.set_init_state(x0)

#set initial control output to be all zeros
us = np.zeros((T+1,sys.Du))
_ = sys.compute_matrices(x0, us[0])
xs = sys.rollout(us[:-1])

#### Plot initial trajectory

#### Set the regularization cost coefficients Q and R 

In [8]:
Q = np.eye(sys.Dx)*0.01 #coefficient for running cost
Qf = np.eye(sys.Dx)*10  #coefficient for terminal cost
R = np.eye(sys.Du)*0.005  #control coefficient
mu = 1e-6              #regularization coefficient

#### Set reference target

In [9]:
x_target = np.array([1, 1, 1, 0,0, 0])

#### Set obstacle

In [10]:
p_obs1 = np.array([0.01, 0., 0.])   #obstacle position
r_obs1 = 0.15 #radius of the obstacle
w_obs1 = 0.   #weight of the obstacle cost
d_margin1 = 0.1 #margin of the obstacle

In [11]:
#### Set workspace bounds
large_num = 1e10
bounds = np.array([[-1,-1,-1, -large_num, -large_num, -large_num],
                   [1,1,1, large_num, large_num, large_num]])
w_bound = 100.

#### Show the setting in pybullet

In [12]:
_,_,obs_id = create_primitives(radius = r_obs1)
p.resetBasePositionAndOrientation(obs_id, p_obs1, (0,0,0,1))
p.resetBasePositionAndOrientation(obj_id, x0[:3], (0,0,0,1))
p.resetBasePositionAndOrientation(target_id, x_target[:3], (0,0,0,1))

## Setup the ILQR & Solve

#### Define the cost

In [14]:
cost_names_dict = {'state':0, 'control':1, 'bound':2, 'obstacle':3}

#The costs consist of: a) state tracking (Q), b) control regularization (R), c) obstacle cost
#Running cost is for the time 0 <= t < T, while terminal cost is for the time t = T
costs = []
for i in range(T):
    runningStateCost = CostModelQuadratic(sys, Q, x_ref = x_target)
    runningControlCost = CostModelQuadratic(sys, None, R)
    runningBoundCost = CostModelBound(sys, bounds, weight=w_bound)
    runningObstacleCost = CostModelCollisionSphere(sys, p_obs1, r_obs1, w_obs = w_obs1, d_margin=d_margin1)    
    runningCost = CostModelSum(sys, [runningStateCost, runningControlCost, runningBoundCost, runningObstacleCost])
    costs += [runningCost]

terminalStateCost = CostModelQuadratic(sys,Qf, x_ref = x_target)
terminalControlCost = CostModelQuadratic(sys, None,R)
terminalBoundCost = CostModelBound(sys, bounds, weight=w_bound)
terminalObstacleCost = CostModelCollisionSphere(sys, p_obs1, r_obs1, w_obs = w_obs1, d_margin=d_margin1)
terminalCost = CostModelSum(sys, [terminalStateCost, terminalControlCost, terminalBoundCost, terminalObstacleCost])

costs += [terminalCost]

#### Construct ILQR

In [15]:
ilqr_cost = ILQR(sys, mu)
ilqr_cost.set_init_state(x0)
ilqr_cost.set_timestep(T)
ilqr_cost.set_cost(costs)
ilqr_cost.set_state(xs, us)    #set initial guess

#### Solve and Plot

In [16]:
n_iter = 100
ilqr_cost.solve(n_iter, method='recursive', cost_thres=1e-3)
xs_batch, us_batch = ilqr_cost.xs, ilqr_cost.us


Cost converges at iteration 47, cannot decrease further


In [17]:
plot_traj(ilqr_cost.xs[:100], obj_id, dt=0.1)

***

## Modifying the problem

#### Set the initial and goal position

In [30]:
x0 = np.ones(Dx)*-1
x0[3:] *= 0
sys.set_init_state(x0)

#set initial control output to be all zeros
us = np.zeros((T+1,sys.Du))
_ = sys.compute_matrices(x0, us[0])
xs = sys.rollout(us[:-1])


x_target = np.array([1, 1,1, 0,0, 0])
#x_target = np.array([0.05, 0, -0.4, 0,0, 0])

#### Set the obstacle position (and possibly radius)

In [37]:
p_obs1 = np.array([0.05, 0., -0.])   #obstacle position
r_obs1 = 0.5
d_margin1 = 0.1
w_obs1 = 100.
p.removeBody(obs_id)
_,_,obs_id = create_primitives(radius = r_obs1, basePosition=p_obs1)

#### Show the setting in pybullet

In [38]:
p.resetBasePositionAndOrientation(obs_id, p_obs1, (0,0,0,1))
p.resetBasePositionAndOrientation(obj_id, x0[:3], (0,0,0,1))
p.resetBasePositionAndOrientation(target_id, x_target[:3], (0,0,0,1))

#### Modify the iLQR accordingly

In [39]:
#change target
for i in range(T+1):
    ilqr_cost.costs[i].costs[0].x_ref = x_target

#change collision cost
for i in range(T+1):
    ilqr_cost.costs[i].costs[-1].p_obs = p_obs1
    ilqr_cost.costs[i].costs[-1].r_obs = r_obs1
    ilqr_cost.costs[i].costs[-1].w_obs = w_obs1
    ilqr_cost.costs[i].costs[-1].d_thres = r_obs1 + d_margin1
    
    
ilqr_cost.set_init_state(x0)
ilqr_cost.set_state(xs, us)    #set initial guess

#### Solve and Plot

In [40]:
n_iter = 100
ilqr_cost.solve(n_iter, method='recursive', cost_thres=1e-6)
xs_batch, us_batch = ilqr_cost.xs, ilqr_cost.us


Reach maximum ILQR iteration


In [46]:
plot_traj(ilqr_cost.xs[:], obj_id, dt=0.1)

#### Check cost

In [47]:
target_thres = 0.1
obstacle_thres = r_obs1 + 0.03

In [48]:
def check_cost(ilqr_cost, target_thres, obstacle_thres, cost_names_dict, T):
    dist_obstacles = np.array([ilqr_cost.costs[i].costs[cost_names_dict['obstacle']].dist for i in range(T+1)])
    collision_status_array = dist_obstacles > obstacle_thres
    collision_status = np.min(collision_status_array)    

    cost_target_res = ilqr_cost.costs[-1].costs[cost_names_dict['state']].res
    target_status = np.linalg.norm(cost_target_res) < target_thres
    
    status = np.logical_and(collision_status, target_status)
    return status, target_status, collision_status, dist_obstacles

In [49]:
status, target_status, collision_status, dist_obstacles = check_cost(ilqr_cost, target_thres, obstacle_thres, cost_names_dict, T)

In [50]:
print(status, target_status, collision_status)

True True True


#### Save Data

In [172]:
data = dict()

In [173]:
data['xs'] = ilqr_cost.xs
data['us'] = ilqr_cost.us
data['x0'] = x0
data['xT'] = x_target
data['p_obs'] = p_obs1
data['r_obs'] = r_obs1
data['status'] = status
data['target_status'] = target_status
data['collision_status'] = collision_status
data['dist_obstacles'] = dist_obstacles
data['target_thres'] = target_thres
data['obstacle_thres'] = obstacle_thres
data['iter'] = ilqr_cost.iter

In [174]:
np.save('data.npy', data)